In [ ]:
from math import exp, sqrt, log
from scipy.stats import norm
import statistics

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, ShuffleSplit
from sklearn import svm
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 1. Make Sample



## 1. 1 Data preprocess

### 1. 1. 1 Create option data

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as si

def N(x): # cdf for standard normal distribution
  return si.norm.cdf(x, 0., 1.)

def european_option(S, K, T, r, d, sigma, call_option = 1):
  # S is spot price
  # K is strike price 
  # T is time to maturity (unit: years)
  # r is risk-free rate per year
  # d is dividend rate per year
  # sigma is volatility per year 
  # call_option = 1 if call 0 if put option 

  F = S*np.exp((r-d)*T) # forward price 
  B = np.exp(-r*T)      # discount factor 
  d1 = (np.log(F/K) + 0.5*sigma**2*T)/(sigma*T**.5)
  d2 = (np.log(F/K) - 0.5*sigma**2*T)/(sigma*T**.5)
  if call_option == 1:
    result = (F*N(d1)-K*N(d2))
  if call_option == 0:
    result = (K*N(-d2)-F*N(-d1))
  return B*result

In [ ]:
# Simulate inputs for option data 
np.random.seed(42)
draws = 10**4 # of simulation
S = np.random.rand(draws)*100.             #is spot price
K = np.random.randint(50,150,draws)*.01*S  #is strike price 
T = np.random.randint(10,300,draws)*.01    #is time to maturity (unit: years)
r = np.random.rand(draws)*.1  #is risk-free rate per year
d = np.random.rand(draws)*.1  #is dividend rate per year
sigma = np.random.rand(draws) #is volatility per year 
opt_type = np.random.choice([0,1],draws)

In [ ]:
# simulate option prices
opt_price = []
for i in range(draws):
  p =  european_option(S[i], K[i], T[i], r[i], d[i], sigma[i], opt_type[i])
  opt_price.append(p)
  if (i % 5000) == 0:
    print('Generated {} options'.format(i))

# create dataframe
options = pd.DataFrame({'S':S,
                        'K':K,
                        'T':T,
                        'r':r,
                        'd':d,
                        'simga':sigma,
                        'type':opt_type,
                        'price':opt_price})

print(options.head())
print(options.shape)

Generated 0 options
Generated 5000 options
           S           K     T  ...     simga  type         price
0  37.454012   38.952172  0.44  ...  0.863077     0  9.346612e+00
1  95.071431  137.853574  0.61  ...  0.873235     0  5.661131e+01
2  73.199394   62.219485  1.70  ...  0.398481     1  1.527382e+01
3  59.865848   51.484630  0.93  ...  0.008923     0  2.554058e-76
4  15.601864    9.829174  0.35  ...  0.186566     0  1.903052e-06

[5 rows x 8 columns]
(10000, 8)


In [ ]:
options

,S,K,T,r,d,simga,type,price
0,37.454012,38.952172,0.44,0.040804,0.061208,0.863077,0,9.346612e+00
1,95.071431,137.853574,0.61,0.003737,0.003619,0.873235,0,5.661131e+01
2,73.199394,62.219485,1.70,0.023706,0.073924,0.398481,1,1.527382e+01
3,59.865848,51.484630,0.93,0.077922,0.071247,0.008923,0,2.554058e-76
4,15.601864,9.829174,0.35,0.060599,0.007961,0.186566,0,1.903052e-06
...,...,...,...,...,...,...,...,...
9995,85.765599,43.740455,1.61,0.086058,0.077362,0.283525,1,3.784575e+01
9996,89.750884,131.036290,0.73,0.067761,0.027069,0.262337,1,6.023531e-01
9997,94.670792,117.391781,1.59,0.020053,0.081181,0.159488,0,3.100699e+01
9998,39.748799,58.430735,0.21,0.007262,0.034786,0.046991,0,1.888221e+01


## 1. 2 Model

- use SVR model

### 1. 2. 1. Define Dataset

In [ ]:
# Define dataset for features (X) and target (y)
X,y = options[options.columns.difference(['price'])], options['price']
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size = .1)

### 1. 2. 2. Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
print(standardScaler.fit(X_train1))
X_train_Scaled1 = standardScaler.transform(X_train1)
X_test_Scaled1 = standardScaler.transform(X_test1)

StandardScaler(copy=True, with_mean=True, with_std=True)


### 1. 2. 3. Modeling

In [ ]:
svr = SVR()
svr.fit(X_train_Scaled1,y_train1)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
initial_prds = svr.predict(X_test_Scaled1)

In [ ]:
RMSE = sqrt(mean_squared_error(y_test1,initial_prds))
R2 = r2_score(y_test1, initial_prds)
print("RMSE:",RMSE,"입니다","\n"+
      "R^2:",R2,"입니다")

RMSE: 3.0766660167476876 입니다 
R^2: 0.9412032044595295 입니다


In [ ]:
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
param_grid = {'C':[25, 50, 100, 200],'gamma': [0.1,0.2,0.4,0.6,0.8,1]}

best_grid1 = GridSearchCV(SVR(), param_grid, refit=True,
                     verbose=2, cv = ss, n_jobs = -1)
best_grid1.fit(X_train_Scaled1,y_train1)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 82.8min finished


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.25, train_size=None),
             error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [25, 50, 100, 200],
                         'gamma': [0.1, 0.2, 0.4, 0.6, 0.8, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [ ]:
best_grid1.best_params_

{'C': 200, 'gamma': 0.2}

In [ ]:
X_test_Scaled1

array([[-3.70788712e-01,  6.48668305e-01,  1.92626714e-01, ...,
        -1.69890188e+00, -1.25881600e+00,  9.94238818e-01],
       [ 2.68639766e+00,  1.73040122e+00, -9.17441719e-01, ...,
        -1.30039378e+00, -1.57192262e+00, -1.00579457e+00],
       [ 1.11204695e+00,  6.68987294e-01,  1.64719832e-03, ...,
        -1.34771826e+00, -1.52926613e+00,  9.94238818e-01],
       ...,
       [-1.40823478e+00, -1.64908062e+00, -1.01293148e+00, ...,
         1.59434421e+00, -5.62637718e-01, -1.00579457e+00],
       [-1.20679032e+00, -1.48828134e+00, -7.98079522e-01, ...,
         3.79320483e-01, -1.22433725e+00,  9.94238818e-01],
       [ 3.09949641e-01,  1.56203558e-01, -1.45457161e+00, ...,
        -9.49241909e-01, -4.32864343e-01,  9.94238818e-01]])

In [ ]:
best_preds1 = best_grid1.predict(X_test_Scaled1)

In [ ]:
RMSE = sqrt(mean_squared_error(y_test1, best_preds1))
R2 = r2_score(y_test1, best_preds1)
print("RMSE:",RMSE,"입니다","\n"+
      "R^2:",R2,"입니다")

RMSE: 0.47261751365658344 입니다 
R^2: 0.9986125652410456 입니다


In [ ]:
result_ = {'model Price' : y_test1,
           'model SVM RBF' : best_preds1,
          }

result_df = pd.DataFrame(result_)
result_df

,model Price,model SVM RBF
4008,23.156118,22.225235
3431,42.567713,41.671426
8167,0.001642,-0.676288
2932,1.109325,1.152498
3792,18.610193,18.787406
...,...,...
8069,17.181178,17.279421
6408,16.804542,16.954111
3223,0.488831,0.538400
1038,0.001107,-0.945754


# 2. Real Data
- Kospi 200(Big) (5월물)
- 2020.04.09 - 2020.04.25 (분봉)
- over volume 5

## 2. 1 Data preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/금융공학/real_kospi200 (2).csv")

In [ ]:
data = data[["price","M","R","Q","S","K","type"]]
data.columns = ["price","M","R","Q","S","K","cp"]

### 2. 2. 2 Find volatility

- get **Implied Volatility** using newton raphson method

In [ ]:
def find_vol(target_value, call_put, S, K, T, r, q):
    # S is underlying asset price (stock price)
    # K is exercise price
    # T is time to maturity (annualized)
    # r is risk free rate (annualized)
    # q is dividend rate (annualized)
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5
    m = 1.0  # learning rate; convergence rate
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price(call_put, S, K, T, r, sigma, q)  # Call or put value
        vega = bs_vega(call_put, S, K, T, r, sigma, q) 
        # derivative of option value
        price = price
        diff = target_value - price  # our root (market value - model vaue)
        print (i, sigma, diff)
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + m*diff/vega  # s <- s + m * f(x) / f'(x)
    # If value wasn't found, return best guess so far
    return sigma

In [ ]:
n = norm.pdf   # probability density function for standard normal dist
N = norm.cdf   # cumulative probability density function for standard n.d
# Below is BSM option price
def bs_price(cp_flag,S,K,T,r,v,q):
    # S is underlying asset price (stock price)
    # K is exercise price
    # T is time to maturity (annualized)
    # r is risk free rate (annualized)
    # q is dividend rate (annualized)
    F = S*exp((r-q)*T) # forward price
    d1 = (log(F/K)+(v*v/2.)*T)/(v*sqrt(T))
    d2 = d1-v*sqrt(T)
    if cp_flag == 'c': # this means call option
        price = exp(-r*T)*(F*N(d1)-K*N(d2))
    else:  # this means put option
        price = exp(-r*T)*(K*N(-d2)-F*N(-d1))
    return price
  
# below is vega, i.e. derivative of option value with respect to vol
def bs_vega(cp_flag,S,K,T,r,v,q):
    F = S*exp((r-q)*T)
    d1 = (log(F/K)+(v*v/2.)*T)/(v*sqrt(T))
    return exp(-r*T)* F * sqrt(T)*n(d1)

- It takes long time

In [ ]:
IV = data.apply(lambda x: find_vol(x.price, x.cp, x.S, x.K, x.M, x.R, x.Q),axis=1)

0 0.5 -11.208645176182205
1 0.19563938146823656 -0.21466329707729415
2 0.189382339375037 -0.0006026988928322652
3 0.18936467104510307 -5.110493361826229e-09
0 0.5 -11.723361293517907
1 0.18105252199029104 -0.28779422841082436
2 0.17243059305988184 -0.0016117376318725007
3 0.17238174785556365 -5.6619182231543164e-08
0 0.5 -12.465881118967157
1 0.1600885411060679 -0.4170781435991149
2 0.14693978594782897 -0.006032845171452195
3 0.14674369976507148 -1.5621967257573033e-06
0 0.5 -12.865239640264175
1 0.14877699998439403 -0.5064629946688921
2 0.13219438138918116 -0.012552135803273856
3 0.1317607709762443 -1.0513253045019155e-05
4 0.13176040718830395 -7.331024676204834e-12
0 0.5 -13.287392376929583
1 0.13648734083927477 -0.6396503431291141
2 0.11423562916985044 -0.03141965827766313
3 0.11301347973130615 -0.00012637896698208806
4 0.11300852386312643 -2.1119799242796944e-09
0 0.5 -13.609337276216454
1 0.12706276396161809 -0.7637511306918707
2 0.09882004210163121 -0.06589556923485773
3 0.095812

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Streaming output truncated to the last 5000 lines.
27 nan nan
28 nan nan
29 nan nan
30 nan nan
31 nan nan
32 nan nan
33 nan nan
34 nan nan
35 nan nan
36 nan nan
37 nan nan
38 nan nan
39 nan nan
40 nan nan
41 nan nan
42 nan nan
43 nan nan
44 nan nan
45 nan nan
46 nan nan
47 nan nan
48 nan nan
49 nan nan
50 nan nan
51 nan nan
52 nan nan
53 nan nan
54 nan nan
55 nan nan
56 nan nan
57 nan nan
58 nan nan
59 nan nan
60 nan nan
61 nan nan
62 nan nan
63 nan nan
64 nan nan
65 nan nan
66 nan nan
67 nan nan
68 nan nan
69 nan nan
70 nan nan
71 nan nan
72 nan nan
73 nan nan
74 nan nan
75 nan nan
76 nan nan
77 nan nan
78 nan nan
79 nan nan
80 nan nan
81 nan nan
82 nan nan
83 nan nan
84 nan nan
85 nan nan
86 nan nan
87 nan nan
88 nan nan
89 nan nan
90 nan nan
91 nan nan
92 nan nan
93 nan nan
94 nan nan
95 nan nan
96 nan nan
97 nan nan
98 nan nan
99 nan nan
0 0.5 -14.297645450537715
1 0.10753947770044142 -1.0397327874099656
2 0.062437998678813826 -0.2809087709581579
3 0.03102959137182809 -0.1717861113

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in double_scalars


Streaming output truncated to the last 5000 lines.
14 nan nan
15 nan nan
16 nan nan
17 nan nan
18 nan nan
19 nan nan
20 nan nan
21 nan nan
22 nan nan
23 nan nan
24 nan nan
25 nan nan
26 nan nan
27 nan nan
28 nan nan
29 nan nan
30 nan nan
31 nan nan
32 nan nan
33 nan nan
34 nan nan
35 nan nan
36 nan nan
37 nan nan
38 nan nan
39 nan nan
40 nan nan
41 nan nan
42 nan nan
43 nan nan
44 nan nan
45 nan nan
46 nan nan
47 nan nan
48 nan nan
49 nan nan
50 nan nan
51 nan nan
52 nan nan
53 nan nan
54 nan nan
55 nan nan
56 nan nan
57 nan nan
58 nan nan
59 nan nan
60 nan nan
61 nan nan
62 nan nan
63 nan nan
64 nan nan
65 nan nan
66 nan nan
67 nan nan
68 nan nan
69 nan nan
70 nan nan
71 nan nan
72 nan nan
73 nan nan
74 nan nan
75 nan nan
76 nan nan
77 nan nan
78 nan nan
79 nan nan
80 nan nan
81 nan nan
82 nan nan
83 nan nan
84 nan nan
85 nan nan
86 nan nan
87 nan nan
88 nan nan
89 nan nan
90 nan nan
91 nan nan
92 nan nan
93 nan nan
94 nan nan
95 nan nan
96 nan nan
97 nan nan
98 nan nan
99 nan nan
0 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in double_scalars


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0 0.5 -27.70015233749492
1 -0.17893519633348276 8.998178718324121
2 0.20967507054031848 -17.082443132991926
3 -0.4083425727693319 16.72810158750223
4 0.018770829041631465 -15.040106402914562
5 -1.8492230387470655e+25 390.5665321961024
6 inf nan
7 nan nan
8 nan nan
9 nan nan
10 nan nan
11 nan nan
12 nan nan
13 nan nan
14 nan nan
15 nan nan
16 nan nan
17 nan nan
18 nan nan
19 nan nan
20 nan nan
21 nan nan
22 nan nan
23 nan nan
24 nan nan
25 nan nan
26 nan nan
27 nan nan
28 nan nan
29 nan nan
30 nan nan
31 nan nan
32 nan nan
33 nan nan
34 nan nan
35 nan nan
36 nan nan
37 nan nan
38 nan nan
39 nan nan
40 nan nan
41 nan nan
42 nan nan
43 nan nan
44 nan nan
45 nan nan
46 nan nan
47 nan nan
48 nan nan
49 nan nan
50 nan nan
51 nan nan
52 nan nan
53 nan nan
54 nan nan
55 nan nan
56 nan nan
57 nan nan
58 nan nan
59 nan nan
60 nan nan
61 nan nan
62 nan nan
63 nan nan
64 nan nan
65 nan nan
66 nan nan
67 nan nan
68 nan nan
69 nan nan
70 nan nan
71 nan nan
72 nan 

In [ ]:
IV = pd.DataFrame(IV)

In [ ]:
real = pd.concat([data,IV],axis=1).dropna()

In [ ]:
real.columns = ["price","T","r","q","S","K","type","sigma"]

In [ ]:
#real.to_csv("/content/gdrive/MyDrive/금융공학/real.csv")
real = pd.read_csv("/content/gdrive/MyDrive/금융공학/real.csv")

## 2. 2 Model

### 2. 2. 1 Define Dataset

In [ ]:
# Define dataset for features (X) and target (y)
X,y = real[real.columns.difference(['price'])], real['price']
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size = .1)

### 2. 2. 2. Scaling

In [ ]:
standardScaler = StandardScaler()
print(standardScaler.fit(X_train2))
X_train_Scaled2 = standardScaler.transform(X_train2)
X_test_Scaled2 = standardScaler.transform(X_test2)

StandardScaler(copy=True, with_mean=True, with_std=True)


### 2. 2. 3. Modeling

In [ ]:
svr2 = SVR()
svr2.fit(X_train_Scaled2,y_train2)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
initial_prds2 = svr2.predict(X_test_Scaled2)

In [ ]:
RMSE = sqrt(mean_squared_error(y_test2,initial_prds2))
R2 = r2_score(y_test2, initial_prds2)
print("RMSE:",RMSE,"입니다","\n"+
      "R^2:",R2,"입니다")

RMSE: 0.11900545383939938 입니다 
R^2: 0.9972258411912995 입니다


In [ ]:
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
param_grid = {'C':[1, 25, 50, 100, 200, 250,300,350],'gamma': [0.001, 0.01, 0.1,0.2]}
#param_grid = {'C':[200, 250],'gamma': [0.2,0.4]}

best_grid2 = GridSearchCV(SVR(epsilon = 0.01), param_grid, refit=True,
                     verbose=2, cv = ss, n_jobs = -1)
best_grid2.fit(X_train_Scaled2,y_train2)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   34.8s finished


GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.25, train_size=None),
             error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.01, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1, 25, 50, 100, 200, 250, 300, 350],
                         'gamma': [0.001, 0.01, 0.1, 0.2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [ ]:
best_grid2.best_params_

{'C': 350, 'gamma': 0.1}

In [ ]:
best_preds2 = best_grid2.predict(X_test_Scaled2)

In [ ]:
RMSE = sqrt(mean_squared_error(y_test2,best_preds2))
R2 = r2_score(y_test2, best_preds2)
print("RMSE:",RMSE,"입니다","\n"+
      "R^2:",R2,"입니다")

RMSE: 0.496506146990375 입니다 
R^2: 0.9517111515975931 입니다


In [ ]:
result_ = {'Real Price' : y_test2,
           'real SVM RBF' : best_preds2,

          }

result_df = pd.DataFrame(result_)
result_df

,Real Price,real SVM RBF
562,5.79,5.785865
357,3.50,3.532329
54,12.25,12.134357
451,5.30,5.213392
62,8.36,12.892722
...,...,...
462,5.53,5.538652
360,3.65,4.060855
144,3.41,3.414984
561,5.81,5.835415
